<a href="https://colab.research.google.com/github/iued-uni-heidelberg/DAAD-Training-2021/blob/main/Terminologieextraktion6EvaluationKeyWordsV01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Terminology extraction with keywords, association measures, etc.

Here we enrich extracted MWEs with keyness information (and other paremeters) and test Precision / Recall

In [ ]:
# Stage 0: Preparing Gold standard: Reading / extracting information from gold standard: creating a list of annotated terms
!wget https://heibox.uni-heidelberg.de/f/ae1110c4f9ad42b9a3d5/?dl=1
!mv index.html?dl=1 BGH1_s00Astghik.txt
!wget https://heibox.uni-heidelberg.de/f/0c787f26123f49178639/?dl=1
!mv index.html?dl=1 BGH1_s00Hayk.txt

In [ ]:
!cat BGH1_s00Astghik.txt BGH1_s00Hayk.txt >BGH1_s00GoldStandard.txt

In [ ]:
FInputKW = open('TK_KW_Verif_V02.csv', 'r')
FOutputGSKWS1w = open('BGH1_s01GoldSKW_D1w.txt', 'w')
FOutputGSKWS2w = open('BGH1_s01GoldSKW_D2w.txt', 'w')
FOutputGSKWS3w = open('BGH1_s01GoldSKW_D3w.txt', 'w')
FOutputGSKWSMWE = open('BGH1_s01GoldSKW_MWE.txt', 'w')

In [ ]:
DScoresKW = {} # keywords - scores
DScoresNK = {} # non-keywords
DStatKW = {} # status: key/non-key-word
for Line in FInputKW:
    LFieldsKW = re.split('\t', Line)
    SWord = LFieldsKW[1]
    AKScore = float(LFieldsKW[2])
    AKStat = float(LFieldsKW[3])
    DStatKW[SWord] = AKStat
    if AKStat > 0: # change value to 0.5 if we need to restrict to 'sure' terms only (value 1)
        DScoresKW[SWord] = AKScore
    else:
        DScoresNK[SWord] = AKScore

In [ ]:
# we create a dictionary of keyness values with only upper case letters, which will be checked against also uppercased term candidates
# the same dictionary as DScoresKW, but with ensured conversion in to upper case:
DScoresREKWquick = {} # dictionary of RE
for kw, val in DScoresKW.items():
    SUpperC = kw.upper() # making sure our key words are in upper case
    # these are the alternatives, which we do not consider in this stage...
    # SLowerC = kw.lower()
    # SSentenceC = kw.capitalize()
    # RPatternKW = re.compile('^' + kw + '$', re.IGNORECASE)
    DScoresREKWquick[SUpperC] = val # 
    # DScoresREKWquick[SLowerC] = val # 
    # DScoresREKWquick[SSentenceC] = val # 

In [ ]:
def printDictionary(DictionaryFrq, FOut, Rev = True):
    for Word, Frq in sorted( DictionaryFrq.items() , key=lambda x: x[1], reverse=Rev):
        FOut.write(Word + '\t' + str(Frq) + '\n')
    return